In [ ]:
On dispose d'un dictionnaire de dictionnaire de dictionnaire,
d'abord de clés : "Four 1", "Four 2", "Four 3", "Four 4".
Puis de clés : "etat_Consom", "etat_Serie", "etat_Panne", "etat_Fin". 
Enfin les clés : "1","2","3","4","5","6".
Les valeurs sont des listes. 

As-tu une meilleur structure pour organiser les données ?


On dispose de 3 dictionnaire de dictionnaire times_dict, PFCvalues_dict, PFCvalues_dict
d'abord de clés : "etat_Consom", "etat_Serie", "etat_Panne", "etat_Fin".
Puis de clés : "courbe 1"," courbe 2","courbe 3","courbe 4","courbe 5","courbe 6". Les valeurs sont des listes. 
Les valeurs sont des listes.

Voici les changement d'etat possible :
-"etat_Consom" à "etat_Serie" et inversement "etat_Serie" à "etat_Consom"
-"etat_Consom" à "etat_Panne" et inversement "etat_Panne" à "etat_Consom"
- "etat_Serie" à "etat_Panne"

A chaque changement d' état, on passe de la courbe i à la  courbe i+1.

Peux- tu completer le code pour qu'il fasse cela ?


# Code 


In [3]:
#### Variables Globales 


# Variables de simulation
PFCmin = 2500 # en kg
PFCmax = 5000 # en kg
Mgmin = 0.035 # en %
Mgmax = 0.045 # en %
eC = 0.0005 # en %
poche_i = 1250 # en kg

temps_serie = 5  # en min
temps_traitement = 10 # en min
temps_serie_courant = temps_serie
numPoche = 3
numFours = 1



# Données initiales
liste_cadences_moule_par_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190]  # en unités/heure
liste_quantites_moules_a_produire = [121, 50, 35, 10, 100, 50, 35, 50, 40]  # en unités
liste_masses_grappes_moules = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65]  # en kg

# Dictionnaires pour stocker les variables et états
cles = ["etat_Consom", "etat_Serie", "etat_Panne", "etat_Fin"]
times_dict = {cle: {} for cle in cles}
PFCvalues_dict = {cle: {} for cle in cles}
Mgvalues_dict = {cle: {} for cle in cles}



debut,fin = 37,37  #37,50
debutA,finA = 38,38 #37,50 38,49
debutB,finB = 37,50


courbe_index = 1
courbeSerie_index = 1
courbeConsom_index = 1
courbePanne_index = 1


etat_courant = "etat_Consom" # État initial : production de moules
PFC = 3750  # Initialisation de la fonte consommée
Mg =  Mgmax  # Initialisation du pourcentage de Mg
t = 0 # en minutes










def calcul_duree_limite_et_delai_traitement(Mg, PFC, etat_courant, temps_serie_courant):
    """
    Cette fonction calcule la durée limite avant l'épuisement de Mg ou de PFC et le délai avant de lancer la prochaine poche de traitement.

    Args:
    Mg (float): Quantité initiale de Mg.
    PFC (float): Quantité initiale de PFC.

    Returns:
    int: DurreLimite, durée limite avant l'épuisement de Mg ou PFC.
    int: delai_avt_traitement, délai avant de lancer la prochaine poche de traitement.
    """

    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire

    # Copie des listes pour éviter les modifications
    masses_grappes = liste_masses_grappes_moules.copy()
    cadences_moule = liste_cadences_moule_par_heure.copy()
    quantites_moules = liste_quantites_moules_a_produire.copy()

    ## on calcule le temps mis avant l'épuisement du MgDispo
    # On calcule le pourcentage de Mg consommable
    MgDispo = Mg - Mgmin
    # On calcule le temps mis avant l'épuisement du MgDispo
    DurreMg = MgDispo/eC

    ## on calcule le temps avant l'épuisement de la fonte (DurrePFC)
    # On calcule a fonte disponible pour la production de Moules
    PFCDispo = PFC - PFCmin
    PFCrestant = PFCDispo

    # Avec le PFCDispo, on veux calculer la durée avant l'épuisement 
    # de la fonte disponible et aussi calcule masse de la fonte consommer
    DurrePFC = 0
    PFCrestant = PFCDispo
    # Tant que l'on a de la fonte disponible
    # while PFCrestant != 0 :
    # while True:
    while cadences_moule :
        masse_grappe_i = masses_grappes[0]
        cadence_moule_par_heure_i = cadences_moule[0]
        quantite_moules_a_produire_i = quantites_moules[0]
        # La cadence de consommation en kg/min du i-ème modèle
        cadence_fonte_i = cadence_moule_par_heure_i / 60 * masse_grappe_i  # en kg/min
        # Quantités de moules du i-ème modèle produits en une minute
        cadence_moule_i = int(cadence_fonte_i / masse_grappe_i) # en unités/min
        # on calcule le nombre de moules du i-ème modèle que l'on pourrait faire 
        # avec cette onte disponible
        nbMoules_i = int(PFCrestant/masse_grappe_i)


        # Si la fonte disponible permet de produire tous les moules du i-èmes modèles
        if nbMoules_i >=  quantite_moules_a_produire_i :
            # Alors on met à jour la fonte total consommer et la durée DurrePFC
            #  après avoir produit tous les moules du i-ème modèles
            PFCrestant -= quantite_moules_a_produire_i* masse_grappe_i # en kg
            DurrePFC += int(quantite_moules_a_produire_i /cadence_moule_i) # en min

            # On change le modèle i par le modèle i+1
            # Pendant ce changement, on atttend temps_Serie minutes
            DurrePFC += temps_serie_courant #temps_serie


            # On passe au modèle i+1, donc on met à jour le programme des moules
            masses_grappes.pop(0)
            cadences_moule.pop(0)
            quantites_moules.pop(0)

        # Si la fonte disponible ne permet pas de produire tous les moules du i-èmes modèles
        else :
            # Alors on met à jour la fonte total consommer et la durée DurrePFC
            # après avoir produit les moules du i-ème modèles que l'on peut faire
            
            PFCrestant -= nbMoules_i* masse_grappe_i # en kg
            DurrePFC += nbMoules_i /cadence_moule_i # en min
            
            if etat_courant == "etat_Serie" :
                # On change le modèle i par le modèle i+1
                # Pendant ce changement, on atttend temps_Serie minutes
                DurrePFC += temps_serie_courant #temps_serie

            # On met à jour le programme de productions des moules
            quantites_moules[0] -= nbMoules_i

            # Meme si le PFCrestant n'est pas exactement nul, 
            # on arrete la production de moules
            break



    # On obient finalement la deadline (le temps dont on dispose)
    # avant que Mg ou PFC devienne insufisant
    DurreLimite = int(min(DurreMg,DurrePFC))  # en min

    # On a aussi le delai dont on dispose avant de 
    # Lancer la prochaine poche de traitement
    delai_avt_traitement = (DurreLimite - temps_traitement)

    return DurreLimite, delai_avt_traitement


def ProduirMoulesParMinute(Mg, PFC, quantite_moules_a_produire_i, cadence_fonte_i, cadence_moule_i):
    
    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement, courbe_index, courbeSerie_index
    # Variables dans ce programme
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire

    # Mise à jour du pourcentage de Mg et du poids fonte coulée
    PFC -= cadence_fonte_i
    Mg -= eC

    # if PFC > PFCmax:
    #     print("On a dépassé la capacité du Four de Coulée")
    

    quantite_moules_restants = quantite_moules_a_produire_i - cadence_moule_i

    if debut < t < fin:
        print("Ici2,t =",t, Mg, PFC, cadence_fonte_i, quantite_moules_a_produire_i, quantite_moules_restants)

    # On a fini de réaliser le i-ème modèle alors on passe au i+1-ème modèle
    if quantite_moules_restants <= 0:
        etat_suivant = "etat_Serie"
        # Suppression des éléments déjà traités dans les listes
        liste_masses_grappes_moules.pop(0)
        liste_quantites_moules_a_produire.pop(0)
        liste_cadences_moule_par_heure.pop(0)

        # Si on a fini de tout produire alors on stoppe la procédure 
        # On passe à l'état Fin
        if not liste_masses_grappes_moules:
            etat_suivant = "etat_Fin"

        courbe_index +=1
        courbeSerie_index += 1
        
        if debut < t < fin:
            print("Ici3,t =",t, Mg, PFC, cadence_fonte_i, quantite_moules_a_produire_i, quantite_moules_restants)

    # On a fini de réaliser le j-ème moule du i-ème modèle alors on passe au j-ème moule
    # En restant dans l'état consommation 
    else:
        if debut < t < fin:
            print("Ici3,t =",t, Mg, PFC, cadence_fonte_i, quantite_moules_a_produire_i, quantite_moules_restants)

        liste_quantites_moules_a_produire[0] = quantite_moules_restants
        etat_suivant= "etat_Consom"
    
    return Mg, PFC, etat_suivant


def gerer_etat(etat_courant, Mg, PFC, Panne):
    """
    Gère les transitions entre différents états et effectue les opérations associées.

    Parameters:
        etat_courant (str): L'état actuel du système ("etat_Consom", "etat_Serie", "etat_Panne", "etat_Fin").
        Mg (float): La quantité de matériau Mg.
        PFC (float): La quantité de PFC disponible.
        Panne (bool): Indique si le système est en panne.

    Returns:
        Tuple: Retourne les variables mises à jour pour Mg, PFC, etat_suivant.
    """

    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement, temps_serie_courant, temps_serie, courbe_index, courbePanne_index, courbeConsom_index, courbeSerie_index
    # Variables dans ce programme
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire


    # Gérer l'état de consommation
    if etat_courant == "etat_Consom":
        # On produit les moules !! 
        masse_grappe_i = liste_masses_grappes_moules[0]
        cadence_moule_par_heure_i = liste_cadences_moule_par_heure[0]
        quantite_moules_a_produire_i = liste_quantites_moules_a_produire[0]
        
        # La cadence de consommation en kg/min du i-ème modèle
        cadence_fonte_i = cadence_moule_par_heure_i / 60 * masse_grappe_i  # en kg/min
        
        # Quantités de moules du i-ème modèle produits en une minute
        cadence_moule_i = int(cadence_fonte_i / masse_grappe_i) # en unités/min

        # Vérification de la période de production
        if debut < t < fin:
            print("Ici1, t =", t, Mg, PFC, cadence_fonte_i, quantite_moules_a_produire_i)
        
        # Si l'on peut produire des moules (c-à-d Mg et PFC sont suffisants)
        if (PFC - cadence_fonte_i >= PFCmin) and (Mg - eC >= Mgmin):
            Mg, PFC, etat_suivant = ProduirMoulesParMinute(
                Mg, PFC, quantite_moules_a_produire_i, cadence_fonte_i, cadence_moule_i
            )
        else:
            etat_suivant = etat_courant  # Reste dans l'état actuel si les conditions ne sont pas remplies

        # Gestion des pannes
        if Panne:
            etat_suivant = "etat_Panne"
            courbePanne_index += 1
            courbe_index += 1

    # Gérer l'état de série
    elif etat_courant == "etat_Serie":
        # Réduction de Mg due à la consommation
        Mg -= eC
        temps_serie_courant -= 1

        # Vérification de la fin de la série
        if temps_serie_courant == 0:
            if debutA < t < finA:
                print("Fin Temps Serie", Mg, PFC, cadence_fonte_i)
            
            temps_serie_courant = temps_serie  # Réinitialiser le temps de série
            etat_suivant = "etat_Consom"
            courbe_index += 1
            courbeConsom_index += 1

        # Gestion des pannes
        elif Panne:
            etat_suivant = "etat_Panne"
            courbe_index += 1
            courbePanne_index += 1
        else:
            etat_suivant = "etat_Serie"  # Reste dans l'état actuel

    # Gérer l'état de panne
    elif etat_courant == "etat_Panne":
        Mg -= eC
        etat_suivant = "etat_Panne"  # Reste en panne
        if debut < t < fin:
            print("Fin Panne", Mg, PFC)

    # Gérer l'état de fin
    elif etat_courant == "etat_Fin":
        etat_suivant = "etat_Fin"  # Fin du processus

    return Mg, PFC, etat_suivant


def update_state_data(etat_courant, t, PFC, Mg ):
    """
    Updates the dictionaries for the current state's curve data.

    Parameters:
        etat_courant (str): The current state of the system ("etat_Consom", "etat_Serie", "etat_Panne").
        t (int): The current time.
        PFC (float): The current PFC value.
        Mg (float): The current Mg value.
        courbeConsom_index (int): The index for the consumption curve.
        courbeSerie_index (int): The index for the series curve.
        courbePanne_index (int): The index for the breakdown curve.
        times_dict (dict): Dictionary storing time data for each state.
        PFCvalues_dict (dict): Dictionary storing PFC values for each state.
        Mgvalues_dict (dict): Dictionary storing Mg values for each state.

    Returns:
        tuple: Updated dictionaries and current curve index.
    """
        # Constantes dans ce programme
    global  times_dict, PFCvalues_dict, Mgvalues_dict, courbe_index, courbePanne_index, courbeConsom_index, courbeSerie_index



    # Determine the current curve index based on the current state
    if etat_courant == "etat_Consom":
        # Ensure keys exist for the consumption state
        times_dict[etat_courant].setdefault(f'courbe {courbeConsom_index}', [])
        PFCvalues_dict[etat_courant].setdefault(f'courbe {courbeConsom_index}', [])
        Mgvalues_dict[etat_courant].setdefault(f'courbe {courbeConsom_index}', [])
        courbe_index = courbeConsom_index
    
    elif etat_courant == "etat_Serie":
        # Ensure keys exist for the series state
        times_dict[etat_courant].setdefault(f'courbe {courbeSerie_index}', [])
        PFCvalues_dict[etat_courant].setdefault(f'courbe {courbeSerie_index}', [])
        Mgvalues_dict[etat_courant].setdefault(f'courbe {courbeSerie_index}', [])
        courbe_index = courbeSerie_index

    elif etat_courant == "etat_Panne":
        # Ensure keys exist for the breakdown state
        times_dict[etat_courant].setdefault(f'courbe {courbePanne_index}', [])
        PFCvalues_dict[etat_courant].setdefault(f'courbe {courbePanne_index}', [])
        Mgvalues_dict[etat_courant].setdefault(f'courbe {courbePanne_index}', [])
        courbe_index = courbePanne_index
    
    else:
        raise ValueError("Invalid etat_courant: must be 'etat_Consom', 'etat_Serie', or 'etat_Panne'.")

    # Ensure the key for the current curve exists
    times_dict[etat_courant].setdefault(f'courbe {courbe_index}', [])
    PFCvalues_dict[etat_courant].setdefault(f'courbe {courbe_index}', [])
    Mgvalues_dict[etat_courant].setdefault(f'courbe {courbe_index}', [])

    # Append the new values to the current state's curve
    times_dict[etat_courant][f'courbe {courbe_index}'].append(t)
    PFCvalues_dict[etat_courant][f'courbe {courbe_index}'].append(PFC)
    Mgvalues_dict[etat_courant][f'courbe {courbe_index}'].append(Mg)

    return times_dict, PFCvalues_dict, Mgvalues_dict



def simulation(etat_courant, t, Mg, PFC):
    """
    Simulate the state transitions and resource management until the final state is reached.

    Parameters:
        etat_courant (str): The initial state of the system.
        t (int): The initial time value.
        Mg (float): The initial Mg value.
        PFC (float): The initial PFC value.

    Returns:
        None
    """

    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement, temps_serie_courant, temps_serie, courbe_index, courbePanne_index, courbeConsom_index, courbeSerie_index
    # Variables dans ce programme
    global  poche_i, Mgmax, PFCmax, times_dict, PFCvalues_dict, Mgvalues_dict


    global  Panne, FinPanne

    # Update the state data
    update_state_data(times_dict, PFCvalues_dict, Mgvalues_dict, etat_courant, t, PFC, Mg)

    # Result of the two buttons: Panne and Fin Panne
    # Determine the next state based on user interaction
    if Panne == True:
        etat_suivant = "etat_Panne"

    if FinPanne == True :
        Panne = False
        FinPanne = False
        courbePanne_index += 1
        etat_suivant = "etat_Consom" 

    # etat_courant == "etat_Fin"


    # Calculate the delay and duration limit for the next treatment
    DurreLimite, delai_avt_traitement = calcul_duree_limite_et_delai_traitement(
        Mg, PFC, etat_courant, temps_serie_courant
    )



    # Determine if it's time to initiate the treatment process
    if delai_avt_traitement == 0:
        if debutA < t < finA:
            print("Lancer le prochain traitement !!", t)

    # Determine if it's time to add the treatment batch
    if DurreLimite == 0:
        if debutA < t < finA:
            print("Ajouter la prochain poche !!", t)

        # Add the treatment batch
        PFC += poche_i
        Mg = Mgmax

        # Ensure PFC does not exceed maximum capacity
        if PFC > PFCmax:
            print("On a depasser pas la capacité du Fours de Coulée", t)


    # Update Mg, PFC, and determine the next state
    Mg, PFC, etat_suivant = gerer_etat(etat_courant, Mg, PFC, Panne)
    t += 1
    etat_precedant = etat_courant
    etat_courant = etat_suivant


    return t, Mg, PFC, etat_suivant 





In [4]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
import asyncio
import time

#### Variables Globales 


# Variables de simulation
PFCmin = 2500 # en kg
PFCmax = 5000 # en kg
Mgmin = 0.035 # en %
Mgmax = 0.045 # en %
eC = 0.0005 # en %
poche_i = 1250 # en kg

temps_serie = 5  # en min
temps_traitement = 10 # en min
temps_serie_courant = temps_serie
numPoche = 3
numFours = 1



# Données initiales
liste_cadences_moule_par_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190]  # en unités/heure
liste_quantites_moules_a_produire = [121, 50, 35, 10, 100, 50, 35, 50, 40]  # en unités
liste_masses_grappes_moules = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65]  # en kg

# Dictionnaires pour stocker les variables et états
cles = ["etat_Consom", "etat_Serie", "etat_Panne", "etat_Fin"]
times_dict = {cle: {} for cle in cles}
PFCvalues_dict = {cle: {} for cle in cles}
Mgvalues_dict = {cle: {} for cle in cles}


def calcul_duree_limite_et_delai_traitement(Mg, PFC, etat_courant, temps_serie_courant):
    """
    Cette fonction calcule la durée limite avant l'épuisement de Mg ou de PFC et le délai avant de lancer la prochaine poche de traitement.

    Args:
    Mg (float): Quantité initiale de Mg.
    PFC (float): Quantité initiale de PFC.

    Returns:
    int: DurreLimite, durée limite avant l'épuisement de Mg ou PFC.
    int: delai_avt_traitement, délai avant de lancer la prochaine poche de traitement.
    """

    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire

    # Copie des listes pour éviter les modifications
    masses_grappes = liste_masses_grappes_moules.copy()
    cadences_moule = liste_cadences_moule_par_heure.copy()
    quantites_moules = liste_quantites_moules_a_produire.copy()

    ## on calcule le temps mis avant l'épuisement du MgDispo
    # On calcule le pourcentage de Mg consommable
    MgDispo = Mg - Mgmin
    # On calcule le temps mis avant l'épuisement du MgDispo
    DurreMg = MgDispo/eC

    ## on calcule le temps avant l'épuisement de la fonte (DurrePFC)
    # On calcule a fonte disponible pour la production de Moules
    PFCDispo = PFC - PFCmin
    PFCrestant = PFCDispo

    # Avec le PFCDispo, on veux calculer la durée avant l'épuisement 
    # de la fonte disponible et aussi calcule masse de la fonte consommer
    DurrePFC = 0
    PFCrestant = PFCDispo
    # Tant que l'on a de la fonte disponible
    # while PFCrestant != 0 :
    # while True:
    while cadences_moule :
        masse_grappe_i = masses_grappes[0]
        cadence_moule_par_heure_i = cadences_moule[0]
        quantite_moules_a_produire_i = quantites_moules[0]
        # La cadence de consommation en kg/min du i-ème modèle
        cadence_fonte_i = cadence_moule_par_heure_i / 60 * masse_grappe_i  # en kg/min
        # Quantités de moules du i-ème modèle produits en une minute
        cadence_moule_i = int(cadence_fonte_i / masse_grappe_i) # en unités/min
        # on calcule le nombre de moules du i-ème modèle que l'on pourrait faire 
        # avec cette onte disponible
        nbMoules_i = int(PFCrestant/masse_grappe_i)


        # Si la fonte disponible permet de produire tous les moules du i-èmes modèles
        if nbMoules_i >=  quantite_moules_a_produire_i :
            # Alors on met à jour la fonte total consommer et la durée DurrePFC
            #  après avoir produit tous les moules du i-ème modèles
            PFCrestant -= quantite_moules_a_produire_i* masse_grappe_i # en kg
            DurrePFC += int(quantite_moules_a_produire_i /cadence_moule_i) # en min

            # On change le modèle i par le modèle i+1
            # Pendant ce changement, on atttend temps_Serie minutes
            DurrePFC += temps_serie_courant #temps_serie


            # On passe au modèle i+1, donc on met à jour le programme des moules
            masses_grappes.pop(0)
            cadences_moule.pop(0)
            quantites_moules.pop(0)

        # Si la fonte disponible ne permet pas de produire tous les moules du i-èmes modèles
        else :
            # Alors on met à jour la fonte total consommer et la durée DurrePFC
            # après avoir produit les moules du i-ème modèles que l'on peut faire
            
            PFCrestant -= nbMoules_i* masse_grappe_i # en kg
            DurrePFC += nbMoules_i /cadence_moule_i # en min
            
            if etat_courant == "etat_Serie" :
                # On change le modèle i par le modèle i+1
                # Pendant ce changement, on atttend temps_Serie minutes
                DurrePFC += temps_serie_courant #temps_serie

            # On met à jour le programme de productions des moules
            quantites_moules[0] -= nbMoules_i

            # Meme si le PFCrestant n'est pas exactement nul, 
            # on arrete la production de moules
            break



    # On obient finalement la deadline (le temps dont on dispose)
    # avant que Mg ou PFC devienne insufisant
    DurreLimite = int(min(DurreMg,DurrePFC))  # en min

    # On a aussi le delai dont on dispose avant de 
    # Lancer la prochaine poche de traitement
    delai_avt_traitement = (DurreLimite - temps_traitement)

    return DurreLimite, delai_avt_traitement


def ProduirMoulesParMinute(Mg, PFC, quantite_moules_a_produire_i, cadence_fonte_i, cadence_moule_i):
    
    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement, courbe_index, courbeSerie_index
    # Variables dans ce programme
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire

    # Mise à jour du pourcentage de Mg et du poids fonte coulée
    PFC -= cadence_fonte_i
    Mg -= eC

    # if PFC > PFCmax:
    #     print("On a dépassé la capacité du Four de Coulée")
    

    quantite_moules_restants = quantite_moules_a_produire_i - cadence_moule_i

    if debut < t < fin:
        print("Ici2,t =",t, Mg, PFC, cadence_fonte_i, quantite_moules_a_produire_i, quantite_moules_restants)

    # On a fini de réaliser le i-ème modèle alors on passe au i+1-ème modèle
    if quantite_moules_restants <= 0:
        etat_suivant = "etat_Serie"
        # Suppression des éléments déjà traités dans les listes
        liste_masses_grappes_moules.pop(0)
        liste_quantites_moules_a_produire.pop(0)
        liste_cadences_moule_par_heure.pop(0)

        # Si on a fini de tout produire alors on stoppe la procédure 
        # On passe à l'état Fin
        if not liste_masses_grappes_moules:
            etat_suivant = "etat_Fin"

        courbe_index +=1
        courbeSerie_index += 1
        
        if debut < t < fin:
            print("Ici3,t =",t, Mg, PFC, cadence_fonte_i, quantite_moules_a_produire_i, quantite_moules_restants)

    # On a fini de réaliser le j-ème moule du i-ème modèle alors on passe au j-ème moule
    # En restant dans l'état consommation 
    else:
        if debut < t < fin:
            print("Ici3,t =",t, Mg, PFC, cadence_fonte_i, quantite_moules_a_produire_i, quantite_moules_restants)

        liste_quantites_moules_a_produire[0] = quantite_moules_restants
        etat_suivant= "etat_Consom"
    
    return Mg, PFC, etat_suivant


def gerer_etat(etat_courant, Mg, PFC, Panne):
    """
    Gère les transitions entre différents états et effectue les opérations associées.

    Parameters:
        etat_courant (str): L'état actuel du système ("etat_Consom", "etat_Serie", "etat_Panne", "etat_Fin").
        Mg (float): La quantité de matériau Mg.
        PFC (float): La quantité de PFC disponible.
        Panne (bool): Indique si le système est en panne.

    Returns:
        Tuple: Retourne les variables mises à jour pour Mg, PFC, etat_suivant.
    """

    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement, temps_serie_courant, temps_serie, courbe_index, courbePanne_index, courbeConsom_index, courbeSerie_index
    # Variables dans ce programme
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire


    # Gérer l'état de consommation
    if etat_courant == "etat_Consom":
        # On produit les moules !! 
        masse_grappe_i = liste_masses_grappes_moules[0]
        cadence_moule_par_heure_i = liste_cadences_moule_par_heure[0]
        quantite_moules_a_produire_i = liste_quantites_moules_a_produire[0]
        
        # La cadence de consommation en kg/min du i-ème modèle
        cadence_fonte_i = cadence_moule_par_heure_i / 60 * masse_grappe_i  # en kg/min
        
        # Quantités de moules du i-ème modèle produits en une minute
        cadence_moule_i = int(cadence_fonte_i / masse_grappe_i) # en unités/min

        # Vérification de la période de production
        if debut < t < fin:
            print("Ici1, t =", t, Mg, PFC, cadence_fonte_i, quantite_moules_a_produire_i)
        
        # Si l'on peut produire des moules (c-à-d Mg et PFC sont suffisants)
        if (PFC - cadence_fonte_i >= PFCmin) and (Mg - eC >= Mgmin):
            Mg, PFC, etat_suivant = ProduirMoulesParMinute(
                Mg, PFC, quantite_moules_a_produire_i, cadence_fonte_i, cadence_moule_i
            )
        else:
            etat_suivant = etat_courant  # Reste dans l'état actuel si les conditions ne sont pas remplies

        # Gestion des pannes
        if Panne:
            etat_suivant = "etat_Panne"
            courbePanne_index += 1
            courbe_index += 1

    # Gérer l'état de série
    elif etat_courant == "etat_Serie":
        # Réduction de Mg due à la consommation
        Mg -= eC
        temps_serie_courant -= 1

        # Vérification de la fin de la série
        if temps_serie_courant == 0:
            if debutA < t < finA:
                print("Fin Temps Serie", Mg, PFC, cadence_fonte_i)
            
            temps_serie_courant = temps_serie  # Réinitialiser le temps de série
            etat_suivant = "etat_Consom"
            courbe_index += 1
            courbeConsom_index += 1

        # Gestion des pannes
        elif Panne:
            etat_suivant = "etat_Panne"
            courbe_index += 1
            courbePanne_index += 1
        else:
            etat_suivant = "etat_Serie"  # Reste dans l'état actuel

    # Gérer l'état de panne
    elif etat_courant == "etat_Panne":
        Mg -= eC
        etat_suivant = "etat_Panne"  # Reste en panne
        if debut < t < fin:
            print("Fin Panne", Mg, PFC)

    # Gérer l'état de fin
    elif etat_courant == "etat_Fin":
        etat_suivant = "etat_Fin"  # Fin du processus

    return Mg, PFC, etat_suivant


def update_state_data(etat_courant, t, PFC, Mg ):
    """
    Updates the dictionaries for the current state's curve data.

    Parameters:
        etat_courant (str): The current state of the system ("etat_Consom", "etat_Serie", "etat_Panne").
        t (int): The current time.
        PFC (float): The current PFC value.
        Mg (float): The current Mg value.
        courbeConsom_index (int): The index for the consumption curve.
        courbeSerie_index (int): The index for the series curve.
        courbePanne_index (int): The index for the breakdown curve.
        times_dict (dict): Dictionary storing time data for each state.
        PFCvalues_dict (dict): Dictionary storing PFC values for each state.
        Mgvalues_dict (dict): Dictionary storing Mg values for each state.

    Returns:
        tuple: Updated dictionaries and current curve index.
    """
        # Constantes dans ce programme
    global  times_dict, PFCvalues_dict, Mgvalues_dict, courbe_index, courbePanne_index, courbeConsom_index, courbeSerie_index



    # Determine the current curve index based on the current state
    if etat_courant == "etat_Consom":
        # Ensure keys exist for the consumption state
        times_dict[etat_courant].setdefault(f'courbe {courbeConsom_index}', [])
        PFCvalues_dict[etat_courant].setdefault(f'courbe {courbeConsom_index}', [])
        Mgvalues_dict[etat_courant].setdefault(f'courbe {courbeConsom_index}', [])
        courbe_index = courbeConsom_index
    
    elif etat_courant == "etat_Serie":
        # Ensure keys exist for the series state
        times_dict[etat_courant].setdefault(f'courbe {courbeSerie_index}', [])
        PFCvalues_dict[etat_courant].setdefault(f'courbe {courbeSerie_index}', [])
        Mgvalues_dict[etat_courant].setdefault(f'courbe {courbeSerie_index}', [])
        courbe_index = courbeSerie_index

    elif etat_courant == "etat_Panne":
        # Ensure keys exist for the breakdown state
        times_dict[etat_courant].setdefault(f'courbe {courbePanne_index}', [])
        PFCvalues_dict[etat_courant].setdefault(f'courbe {courbePanne_index}', [])
        Mgvalues_dict[etat_courant].setdefault(f'courbe {courbePanne_index}', [])
        courbe_index = courbePanne_index
    
    else:
        raise ValueError("Invalid etat_courant: must be 'etat_Consom', 'etat_Serie', or 'etat_Panne'.")

    # Ensure the key for the current curve exists
    times_dict[etat_courant].setdefault(f'courbe {courbe_index}', [])
    PFCvalues_dict[etat_courant].setdefault(f'courbe {courbe_index}', [])
    Mgvalues_dict[etat_courant].setdefault(f'courbe {courbe_index}', [])

    # Append the new values to the current state's curve
    times_dict[etat_courant][f'courbe {courbe_index}'].append(t)
    PFCvalues_dict[etat_courant][f'courbe {courbe_index}'].append(PFC)
    Mgvalues_dict[etat_courant][f'courbe {courbe_index}'].append(Mg)

    return times_dict, PFCvalues_dict, Mgvalues_dict




debut,fin = 37,37  #37,50
debutA,finA = 38,38 #37,50 38,49
debutB,finB = 37,50



In [5]:

etat_courant = "etat_Consom" # État initial : production de moules
PFC = 3750  # Initialisation de la fonte consommée
Mg =  Mgmax  # Initialisation du pourcentage de Mg
t = 0 # en minutes

courbe_index = 1
courbeSerie_index = 1
courbeConsom_index = 1
courbePanne_index = 1



def initialize_variables(initial_time_step, initial_t):
    """Initialise les variables de contrôle avec des paramètres."""
    global running, time_step, timedata, PFCdata, Mgdata, t
    running = False
    time_step = initial_time_step
    timedata = []
    PFCdata = []
    Mgdata = []
    t = initial_t

def create_figure():
    """Crée et configure la figure Plotly."""
    fig = go.FigureWidget()
    scatter_PFC = fig.add_scatter(name='PFC', mode='lines+markers').data[0]
    scatter_Mg = fig.add_scatter(name='Mg', mode='lines+markers').data[0]
    fig.update_layout(title="Courbe Interactive", xaxis_title="Temps (minutes)", yaxis_title="Valeur")
    return fig, scatter_PFC, scatter_Mg



async def simulation(etat_courant, t, Mg, PFC):
    """
    Simulate the state transitions and resource management.

    Parameters:
        etat_courant (str): The current state of the system.
        t (int): The current time.
        Mg (float): The current Mg value.
        PFC (float): The current PFC value.

    Returns:
        tuple: Updated time, Mg, PFC, and next state.
    """
    global courbe_index, courbePanne_index, Panne

    # Update state data
    update_state_data(etat_courant, t, PFC, Mg)

    # Handle the result of the "Panne" and "Fin Panne" buttons
    if Panne:
        etat_suivant = "etat_Panne"
    else:
        courbe_index += 1
        courbePanne_index += 1
        etat_suivant = "etat_Consom"

    # Calculate delay and duration limits for treatment
    DurreLimite, delai_avt_traitement = calcul_duree_limite_et_delai_traitement(
        Mg, PFC, etat_courant, temps_serie_courant
    )

    # Print diagnostic information if within the specified period
    if debutA < t < finA:
        print(
            "DurreLimite, delai_avt_traitement, etat_courant, Mg, PFC",
            DurreLimite,
            delai_avt_traitement,
            etat_courant,
            Mg,
            PFC,
        )

    # Determine if it's time to initiate the treatment process
    if delai_avt_traitement == 0:
        if debutA < t < finA:
            print("Lancer le prochain traitement !!", t)

    # Determine if it's time to add the treatment batch
    if DurreLimite == 0:
        if debutA < t < finA:
            print("Ajouter la prochain poche !!", t)

        # Add the treatment batch
        PFC += poche_i
        Mg = Mgmax

        # Ensure PFC does not exceed maximum capacity
        if PFC > PFCmax:
            print("On a depasser pas la capacité du Fours de Coulée", t)

    if debutA < t < finA:
        print("etat_courant,t =", etat_courant, t, Mg, PFC)

    # Update Mg, PFC, and determine the next state
    Mg, PFC, etat_suivant = gerer_etat(etat_courant, Mg, PFC, Panne)
    t += 1

    if debutA < t < finA:
        print("etat_suivant,t =", etat_suivant, t, Mg, PFC)

    return t, Mg, PFC, etat_suivant

async def update_data():
    """Fonction de mise à jour des données."""
    global running, time_step, timedata, PFCdata, Mgdata, t, etat_courant, Mgmax

    # Initial values
    PFC = 3750  # Initialisation de la fonte consommée
    Mg = Mgmax  # Initialisation du pourcentage de Mg
    t = 0  # en minutes
    etat_courant = "etat_Consom"

    while running and etat_courant != "etat_Fin":
        t, Mg, PFC, etat_courant = await simulation(etat_courant, t, Mg, PFC)
        
        # Update data
        timedata.append(t)
        PFCdata.append(PFC)
        Mgdata.append(Mg)

        # Update plot
        with fig.batch_update():
            scatter_PFC.update(x=timedata, y=PFCdata)
            scatter_Mg.update(x=timedata, y=Mgdata)
        
        # Sleep for the time_step duration
        await asyncio.sleep(time_step)




def start_or_next_function(button):
    """Fonction pour démarrer ou changer la fonction."""
    global running
    if not running:
        running = True
        loop = asyncio.get_event_loop()
        loop.create_task(update_data())

def stop_plot(button):
    """Fonction pour arrêter la mise à jour."""
    global running
    running = False

def reset_plot(button):
    """Fonction pour réinitialiser les données et le temps t."""
    global running, timedata, PFCdata, Mgdata, t
    running = False
    timedata = []
    PFCdata = []
    Mgdata = []
    t = 0
    with fig.batch_update():
        scatter_PFC.update(x=timedata, y=PFCdata)
        scatter_Mg.update(x=timedata, y=Mgdata)

def panne_plot(button):
    """Fonction pour activer la panne."""
    global Panne
    Panne = True

def Finpanne_plot(button):
    """Fonction pour arrêter la panne."""
    global Panne
    Panne = False

def create_buttons():
    """Crée les boutons interactifs."""
    start_button = widgets.Button(description="Start/Next Function")
    stop_button = widgets.Button(description="Stop")
    reset_button = widgets.Button(description="Reset")
    panne_button = widgets.Button(description="Panne")
    Finpanne_button = widgets.Button(description="Fin Panne")
    
    panne_button.on_click(panne_plot)
    Finpanne_button.on_click(Finpanne_plot)
    start_button.on_click(start_or_next_function)
    stop_button.on_click(stop_plot)
    reset_button.on_click(reset_plot)

    display(widgets.HBox([start_button, stop_button, reset_button, panne_button, Finpanne_button]), fig)

def main(initial_time_step=1, initial_t=0):
    """Fonction principale pour démarrer l'application."""
    initialize_variables(initial_time_step, initial_t)
    global fig, scatter_PFC, scatter_Mg
    fig, scatter_PFC, scatter_Mg = create_figure()
    create_buttons()

# Appel de la fonction principale pour démarrer l'application
main()


FigureWidget({
    'data': [{'mode': 'lines+markers', 'name': 'PFC', 'type': 'scatter', 'uid': 'f9ec2c91-a2de-4ed7-8a4f-40b9d99897fc'},
             {'mode': 'lines+markers', 'name': 'Mg', 'type': 'scatter', 'uid': '61dfbbed-acff-4ffa-9f58-cde22d703399'}],
    'layout': {'template': '...',
               'title': {'text': 'Courbe Interactive'},
               'xaxis': {'title': {'text': 'Temps (minutes)'}},
               'yaxis': {'title': {'text': 'Valeur'}}}
})

# Code b

In [15]:
# Données initiales
liste_cadences_moule_par_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190]  # en unités/heure
liste_quantites_moules_a_produire = [121, 50, 35, 10, 100, 50, 35, 50, 40]  # en unités
liste_masses_grappes_moules = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65]  # en kg

# Dictionnaires pour stocker les variables et états
cles = ["etat_Consom", "etat_Serie", "etat_Panne", "etat_Fin"]
times_dict = {cle: {} for cle in cles}
PFCvalues_dict = {cle: {} for cle in cles}
Mgvalues_dict = {cle: {} for cle in cles}



PFCmax = 5000 # en kg
PFC = 3750  # Initialisation de la fonte consommée
Mg =  Mgmax  # Initialisation du pourcentage de Mg
t = 0 # en minutes


temps_serie = 5  # en min
temps_serie_courant = temps_serie
numPoche = 3
numFours = 1

tmax = 400
debut,fin = 37,37  #37,50



debutA,finA = 38,38 #37,50 38,49
debutB,finB = 37,50


courbe_index = 1
courbeSerie_index = 1
courbeConsom_index = 1
courbePanne_index = 1


etat_courant = "etat_Consom" # État initial : production de moules





def calcul_duree_limite_et_delai_traitement(Mg, PFC, etat_courant, temps_serie_courant):
    """
    Cette fonction calcule la durée limite avant l'épuisement de Mg ou de PFC et le délai avant de lancer la prochaine poche de traitement.

    Args:
    Mg (float): Quantité initiale de Mg.
    PFC (float): Quantité initiale de PFC.

    Returns:
    int: DurreLimite, durée limite avant l'épuisement de Mg ou PFC.
    int: delai_avt_traitement, délai avant de lancer la prochaine poche de traitement.
    """

    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire

    # Copie des listes pour éviter les modifications
    masses_grappes = liste_masses_grappes_moules.copy()
    cadences_moule = liste_cadences_moule_par_heure.copy()
    quantites_moules = liste_quantites_moules_a_produire.copy()

    ## on calcule le temps mis avant l'épuisement du MgDispo
    # On calcule le pourcentage de Mg consommable
    MgDispo = Mg - Mgmin
    # On calcule le temps mis avant l'épuisement du MgDispo
    DurreMg = MgDispo/eC

    ## on calcule le temps avant l'épuisement de la fonte (DurrePFC)
    # On calcule a fonte disponible pour la production de Moules
    PFCDispo = PFC - PFCmin
    PFCrestant = PFCDispo

    # Avec le PFCDispo, on veux calculer la durée avant l'épuisement 
    # de la fonte disponible et aussi calcule masse de la fonte consommer
    DurrePFC = 0
    PFCrestant = PFCDispo
    # Tant que l'on a de la fonte disponible
    # while PFCrestant != 0 :
    # while True:
    while cadences_moule :
        masse_grappe_i = masses_grappes[0]
        cadence_moule_par_heure_i = cadences_moule[0]
        quantite_moules_a_produire_i = quantites_moules[0]
        # La cadence de consommation en kg/min du i-ème modèle
        cadence_fonte_i = cadence_moule_par_heure_i / 60 * masse_grappe_i  # en kg/min
        # Quantités de moules du i-ème modèle produits en une minute
        cadence_moule_i = int(cadence_fonte_i / masse_grappe_i) # en unités/min
        # on calcule le nombre de moules du i-ème modèle que l'on pourrait faire 
        # avec cette onte disponible
        nbMoules_i = int(PFCrestant/masse_grappe_i)


        # Si la fonte disponible permet de produire tous les moules du i-èmes modèles
        if nbMoules_i >=  quantite_moules_a_produire_i :
            # Alors on met à jour la fonte total consommer et la durée DurrePFC
            #  après avoir produit tous les moules du i-ème modèles
            PFCrestant -= quantite_moules_a_produire_i* masse_grappe_i # en kg
            DurrePFC += int(quantite_moules_a_produire_i /cadence_moule_i) # en min

            # On change le modèle i par le modèle i+1
            # Pendant ce changement, on atttend temps_Serie minutes
            DurrePFC += temps_serie_courant #temps_serie


            # On passe au modèle i+1, donc on met à jour le programme des moules
            masses_grappes.pop(0)
            cadences_moule.pop(0)
            quantites_moules.pop(0)

        # Si la fonte disponible ne permet pas de produire tous les moules du i-èmes modèles
        else :
            # Alors on met à jour la fonte total consommer et la durée DurrePFC
            # après avoir produit les moules du i-ème modèles que l'on peut faire
            
            PFCrestant -= nbMoules_i* masse_grappe_i # en kg
            DurrePFC += nbMoules_i /cadence_moule_i # en min
            
            if etat_courant == "etat_Serie" :
                # On change le modèle i par le modèle i+1
                # Pendant ce changement, on atttend temps_Serie minutes
                DurrePFC += temps_serie_courant #temps_serie

            # On met à jour le programme de productions des moules
            quantites_moules[0] -= nbMoules_i

            # Meme si le PFCrestant n'est pas exactement nul, 
            # on arrete la production de moules
            break



    # On obient finalement la deadline (le temps dont on dispose)
    # avant que Mg ou PFC devienne insufisant
    DurreLimite = int(min(DurreMg,DurrePFC))  # en min

    # On a aussi le delai dont on dispose avant de 
    # Lancer la prochaine poche de traitement
    delai_avt_traitement = (DurreLimite - temps_traitement)

    return DurreLimite, delai_avt_traitement


def ProduirMoulesParMinute(Mg, PFC, quantite_moules_a_produire_i, cadence_fonte_i, cadence_moule_i):
    
    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement, courbe_index, courbeSerie_index
    # Variables dans ce programme
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire

    # Mise à jour du pourcentage de Mg et du poids fonte coulée
    PFC -= cadence_fonte_i
    Mg -= eC

    # if PFC > PFCmax:
    #     print("On a dépassé la capacité du Four de Coulée")
    

    quantite_moules_restants = quantite_moules_a_produire_i - cadence_moule_i

    if debut < t < fin:
        print("Ici2,t =",t, Mg, PFC, cadence_fonte_i, quantite_moules_a_produire_i, quantite_moules_restants)

    # On a fini de réaliser le i-ème modèle alors on passe au i+1-ème modèle
    if quantite_moules_restants <= 0:
        etat_suivant = "etat_Serie"
        # Suppression des éléments déjà traités dans les listes
        liste_masses_grappes_moules.pop(0)
        liste_quantites_moules_a_produire.pop(0)
        liste_cadences_moule_par_heure.pop(0)

        # Si on a fini de tout produire alors on stoppe la procédure 
        # On passe à l'état Fin
        if not liste_masses_grappes_moules:
            etat_suivant = "etat_Fin"

        courbe_index +=1
        courbeSerie_index += 1
        
        if debut < t < fin:
            print("Ici3,t =",t, Mg, PFC, cadence_fonte_i, quantite_moules_a_produire_i, quantite_moules_restants)

    # On a fini de réaliser le j-ème moule du i-ème modèle alors on passe au j-ème moule
    # En restant dans l'état consommation 
    else:
        if debut < t < fin:
            print("Ici3,t =",t, Mg, PFC, cadence_fonte_i, quantite_moules_a_produire_i, quantite_moules_restants)

        liste_quantites_moules_a_produire[0] = quantite_moules_restants
        etat_suivant= "etat_Consom"
    
    return Mg, PFC, etat_suivant


def gerer_etat(etat_courant, Mg, PFC, Panne):
    """
    Gère les transitions entre différents états et effectue les opérations associées.

    Parameters:
        etat_courant (str): L'état actuel du système ("etat_Consom", "etat_Serie", "etat_Panne", "etat_Fin").
        Mg (float): La quantité de matériau Mg.
        PFC (float): La quantité de PFC disponible.
        Panne (bool): Indique si le système est en panne.

    Returns:
        Tuple: Retourne les variables mises à jour pour Mg, PFC, etat_suivant.
    """

    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_traitement, temps_serie_courant, temps_serie, courbe_index, courbePanne_index, courbeConsom_index, courbeSerie_index
    # Variables dans ce programme
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire


    # Gérer l'état de consommation
    if etat_courant == "etat_Consom":
        # On produit les moules !! 
        masse_grappe_i = liste_masses_grappes_moules[0]
        cadence_moule_par_heure_i = liste_cadences_moule_par_heure[0]
        quantite_moules_a_produire_i = liste_quantites_moules_a_produire[0]
        
        # La cadence de consommation en kg/min du i-ème modèle
        cadence_fonte_i = cadence_moule_par_heure_i / 60 * masse_grappe_i  # en kg/min
        
        # Quantités de moules du i-ème modèle produits en une minute
        cadence_moule_i = int(cadence_fonte_i / masse_grappe_i) # en unités/min

        # Vérification de la période de production
        if debut < t < fin:
            print("Ici1, t =", t, Mg, PFC, cadence_fonte_i, quantite_moules_a_produire_i)
        
        # Si l'on peut produire des moules (c-à-d Mg et PFC sont suffisants)
        if (PFC - cadence_fonte_i >= PFCmin) and (Mg - eC >= Mgmin):
            Mg, PFC, etat_suivant = ProduirMoulesParMinute(
                Mg, PFC, quantite_moules_a_produire_i, cadence_fonte_i, cadence_moule_i
            )
        else:
            etat_suivant = etat_courant  # Reste dans l'état actuel si les conditions ne sont pas remplies

        # Gestion des pannes
        if Panne:
            etat_suivant = "etat_Panne"
            courbePanne_index += 1
            courbe_index += 1

    # Gérer l'état de série
    elif etat_courant == "etat_Serie":
        # Réduction de Mg due à la consommation
        Mg -= eC
        temps_serie_courant -= 1

        # Vérification de la fin de la série
        if temps_serie_courant == 0:
            if debutA < t < finA:
                print("Fin Temps Serie", Mg, PFC, cadence_fonte_i)
            
            temps_serie_courant = temps_serie  # Réinitialiser le temps de série
            etat_suivant = "etat_Consom"
            courbe_index += 1
            courbeConsom_index += 1

        # Gestion des pannes
        elif Panne:
            etat_suivant = "etat_Panne"
            courbe_index += 1
            courbePanne_index += 1
        else:
            etat_suivant = "etat_Serie"  # Reste dans l'état actuel

    # Gérer l'état de panne
    elif etat_courant == "etat_Panne":
        Mg -= eC
        etat_suivant = "etat_Panne"  # Reste en panne
        if debut < t < fin:
            print("Fin Panne", Mg, PFC)

    # Gérer l'état de fin
    elif etat_courant == "etat_Fin":
        etat_suivant = "etat_Fin"  # Fin du processus

    return Mg, PFC, etat_suivant




def update_state_data(etat_courant, t, PFC, Mg ):
    """
    Updates the dictionaries for the current state's curve data.

    Parameters:
        etat_courant (str): The current state of the system ("etat_Consom", "etat_Serie", "etat_Panne").
        t (int): The current time.
        PFC (float): The current PFC value.
        Mg (float): The current Mg value.
        courbeConsom_index (int): The index for the consumption curve.
        courbeSerie_index (int): The index for the series curve.
        courbePanne_index (int): The index for the breakdown curve.
        times_dict (dict): Dictionary storing time data for each state.
        PFCvalues_dict (dict): Dictionary storing PFC values for each state.
        Mgvalues_dict (dict): Dictionary storing Mg values for each state.

    Returns:
        tuple: Updated dictionaries and current curve index.
    """
        # Constantes dans ce programme
    global  times_dict, PFCvalues_dict, Mgvalues_dict, courbe_index, courbePanne_index, courbeConsom_index, courbeSerie_index



    # Determine the current curve index based on the current state
    if etat_courant == "etat_Consom":
        # Ensure keys exist for the consumption state
        times_dict[etat_courant].setdefault(f'courbe {courbeConsom_index}', [])
        PFCvalues_dict[etat_courant].setdefault(f'courbe {courbeConsom_index}', [])
        Mgvalues_dict[etat_courant].setdefault(f'courbe {courbeConsom_index}', [])
        courbe_index = courbeConsom_index
    
    elif etat_courant == "etat_Serie":
        # Ensure keys exist for the series state
        times_dict[etat_courant].setdefault(f'courbe {courbeSerie_index}', [])
        PFCvalues_dict[etat_courant].setdefault(f'courbe {courbeSerie_index}', [])
        Mgvalues_dict[etat_courant].setdefault(f'courbe {courbeSerie_index}', [])
        courbe_index = courbeSerie_index

    elif etat_courant == "etat_Panne":
        # Ensure keys exist for the breakdown state
        times_dict[etat_courant].setdefault(f'courbe {courbePanne_index}', [])
        PFCvalues_dict[etat_courant].setdefault(f'courbe {courbePanne_index}', [])
        Mgvalues_dict[etat_courant].setdefault(f'courbe {courbePanne_index}', [])
        courbe_index = courbePanne_index
    
    else:
        raise ValueError("Invalid etat_courant: must be 'etat_Consom', 'etat_Serie', or 'etat_Panne'.")

    # Ensure the key for the current curve exists
    times_dict[etat_courant].setdefault(f'courbe {courbe_index}', [])
    PFCvalues_dict[etat_courant].setdefault(f'courbe {courbe_index}', [])
    Mgvalues_dict[etat_courant].setdefault(f'courbe {courbe_index}', [])

    # Append the new values to the current state's curve
    times_dict[etat_courant][f'courbe {courbe_index}'].append(t)
    PFCvalues_dict[etat_courant][f'courbe {courbe_index}'].append(PFC)
    Mgvalues_dict[etat_courant][f'courbe {courbe_index}'].append(Mg)

    return times_dict, PFCvalues_dict, Mgvalues_dict


# Boucle de simulation
while etat_courant != "etat_Fin":

    # Stocke les donnés dans des dictionnaires
    times_dict, PFCvalues_dict, Mgvalues_dict = update_state_data(etat_courant, t, PFC, Mg)


    # Les resultats des 2 boutons :  Panne et Fin Panne !!!
    Panne = False

    # Si l'utilisateur clique sur le bouton "Debut Panne"
    ## Alors on est dans etat_courant == "etat_Panne" 
    if Panne :
        etat_suivant = "etat_Panne"
    
    # Si l'utilisateur clique sur le bouton "Fin Panne"
    ## Alors on est dans etat_courant == "etat_Consom" 
    else :
        courbe_index +=1
        courbePanne_index += 1
        etat_suivant = "etat_Consom"


    # On calcule le delai du prochain Traitement achat instant
    DurreLimite, delai_avt_traitement = calcul_duree_limite_et_delai_traitement(Mg, PFC,etat_courant, temps_serie_courant)
    
    if  debutA < t and  t < finA :
        print("DurreLimite, delai_avt_traitement, etat_courant, Mg, PFC",DurreLimite, delai_avt_traitement,etat_courant,Mg, PFC)
    

    # Si c'est le moment d'annoncer le lancer la Poche de Traitement
    if delai_avt_traitement == 0 :
        # pass
        # alors on le dit 
        if  debutA < t and  t < finA :
            print("Lancer le prochain traitement !!",t)

    # Si c'est le moment d'ajouter la Poche de Traitement
    if DurreLimite == 0 :
        if  debutA < t and  t < finA :
            print("Ajouter la prochain poche !!",t)
        
        # alors on l'ajoute
        PFC += poche_i
        Mg = Mgmax
    
        if PFC > PFCmax :
            print("On a depasser pas la capacité du Fours de Coulée",t) 


    if  debutA < t and  t < finA :
        print("etat_courant,t =",etat_courant,t, Mg, PFC)


    Mg, PFC, etat_suivant = gerer_etat(etat_courant,Mg,PFC,Panne)
    t += 1
    etat_precedant = etat_courant
    etat_courant = etat_suivant


    if  debutA < t and  t < finA :
        print("etat_suivant,t =",etat_suivant,t, Mg, PFC)
    



In [6]:
liste_masses_grappes_moules,liste_quantites_moules_a_produire

([], [])

In [12]:
print(PFCvalues_dict["etat_Serie"]) # "etat_Serie" "etat_Consom"

{'courbe 2': [3400.8666666666654, 3400.8666666666654, 3400.8666666666654, 3400.8666666666654, 3400.8666666666654], 'courbe 3': [2531.9866666666676, 2531.9866666666676, 2531.9866666666676, 2531.9866666666676, 2531.9866666666676], 'courbe 4': [3189.1866666666665, 3189.1866666666665, 3189.1866666666665, 3189.1866666666665, 3189.1866666666665], 'courbe 5': [4160.026666666667, 4160.026666666667, 4160.026666666667, 4160.026666666667, 4160.026666666667], 'courbe 6': [3474.159999999999, 3474.159999999999, 3474.159999999999, 3474.159999999999, 3474.159999999999], 'courbe 7': [2605.280000000001, 2605.280000000001, 2605.280000000001, 2605.280000000001, 2605.280000000001], 'courbe 8': [3262.48, 3262.48, 3262.48, 3262.48, 3262.48], 'courbe 9': [3544.546666666666, 3544.546666666666, 3544.546666666666, 3544.546666666666, 3544.546666666666]}


In [13]:
print(PFCvalues_dict["etat_Consom"]) # "etat_Serie" "etat_Consom"


{'courbe 1': [3750, 3619.5333333333333, 3489.0666666666666, 3358.6, 3228.133333333333, 3097.6666666666665, 2967.2, 2836.733333333333, 2706.2666666666664, 2575.7999999999997, 3695.333333333333, 3564.8666666666663, 3434.3999999999996, 3303.933333333333, 3173.4666666666662, 3042.9999999999995, 2912.533333333333, 2782.066666666666, 2651.5999999999995, 2521.1333333333328, 3640.666666666666, 3510.1999999999994, 3379.7333333333327, 3249.266666666666, 3118.7999999999993, 2988.3333333333326, 2857.866666666666, 2727.399999999999, 2596.9333333333325, 3716.466666666666, 3585.999999999999, 3455.5333333333324, 3325.0666666666657, 3194.599999999999, 3064.1333333333323, 2933.6666666666656, 2803.199999999999, 2672.733333333332, 2542.2666666666655, 3661.799999999999, 3531.333333333332], 'courbe 2': [3400.8666666666654, 3276.2266666666656, 3151.5866666666657, 3026.946666666666, 2902.306666666666, 2777.666666666666, 2653.026666666666, 2528.3866666666663, 3653.7466666666664, 3529.1066666666666, 3404.466666

In [14]:
print(times_dict["etat_Consom"]) # "etat_Serie" "etat_Consom"

{'courbe 1': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40], 'courbe 2': [46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62], 'courbe 3': [68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79], 'courbe 4': [85, 86, 87, 88], 'courbe 5': [94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127], 'courbe 6': [133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149], 'courbe 7': [155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166], 'courbe 8': [172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188], 'courbe 9': [194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207]}


In [15]:
print(times_dict["etat_Serie"]) # "etat_Serie" "etat_Consom"


{'courbe 2': [41, 42, 43, 44, 45], 'courbe 3': [63, 64, 65, 66, 67], 'courbe 4': [80, 81, 82, 83, 84], 'courbe 5': [89, 90, 91, 92, 93], 'courbe 6': [128, 129, 130, 131, 132], 'courbe 7': [150, 151, 152, 153, 154], 'courbe 8': [167, 168, 169, 170, 171], 'courbe 9': [189, 190, 191, 192, 193]}


In [18]:
import plotly.graph_objects as go

# Initialization of keys
cles = ["etat_Consom", "etat_Serie", "etat_Panne", "etat_Fin"]

# Dictionary of colors for each key
color_dict = {
    "etat_Consom": 'royalblue',
    "etat_Serie": 'firebrick',
    "etat_Panne": 'green',
    "etat_Fin": 'orange'
}

def Plot_consom(PFCvalues_dict, times_dict, PFCmin, PFCmax):
    seen_names = set()
    
    # Create Plotly figure for PFC
    fig_pfc = go.Figure()

    # Iterate through each key in PFCvalues_dict and times_dict
    for key in PFCvalues_dict.keys():
        if key in times_dict:
            for courbe, time_values in times_dict[key].items():
                PFC_values = PFCvalues_dict[key].get(courbe, [])
                trace_name = f"{key}"

                # Add trace only if name is unique
                if trace_name not in seen_names:
                    fig_pfc.add_trace(go.Scatter(x=time_values, y=PFC_values, mode='lines', name=trace_name, line=dict(color=color_dict[key])))
                    seen_names.add(trace_name)
                else:
                    fig_pfc.add_trace(go.Scatter(x=time_values, y=PFC_values, mode='lines', name=trace_name, line=dict(color=color_dict[key]),showlegend=False))
                

    # Add horizontal lines for PFCmin and PFCmax
    fig_pfc.add_shape(
        go.layout.Shape(type="line", x0=0, x1=1, y0=PFCmin, y1=PFCmin, yref='y',
            xref='paper',line=dict(color='red', dash='dash')))
    
    fig_pfc.add_shape(go.layout.Shape(type="line",x0=0,x1=1,y0=PFCmax,y1=PFCmax,
        yref='y',xref='paper',line=dict(color='green', dash='dash')))

    # Customize layout for PFC figure
    fig_pfc.update_layout(
        title='Evolution de la fonte dans le Four de Coulée au cours du temps (PFC)', 
        xaxis_title='Time (minutes)', 
        yaxis_title='PFC (kg)', 
        template='plotly', showlegend=True,
        annotations=[
            dict(x=1, y=PFCmin, xref="paper", yref="y", text="PFCmin", showarrow=True, arrowhead=2, ax=20, ay=-30, font=dict(color='red')),
            dict(x=1, y=PFCmax, xref="paper", yref="y", text="PFCmax", showarrow=True, arrowhead=2, ax=-20, ay=-30, font=dict(color='green'))
        ]
    )

    # Show the PFC plot
    fig_pfc.show()


Plot_consom(PFCvalues_dict, times_dict, PFCmin, PFCmax)


# Courbe interactive

In [2]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
import asyncio
import time

def initialize_variables(initial_time_step, initial_t):
    """Initialise les variables de contrôle avec des paramètres."""
    global running, current_function, time_step, timedata, PFCdata, t
    running = False
    current_function = 0
    time_step = initial_time_step
    timedata = []
    PFCdata = []
    t = initial_t
def create_figure():
    """Crée et configure la figure Plotly."""
    fig = go.FigureWidget()
    scatter = fig.add_scatter(mode='lines+markers').data[0]
    fig.update_layout(title="Courbe Interactive", xaxis_title="Temps", yaxis_title="Valeur")
    return fig, scatter

async def update_data():
    """Fonction de mise à jour des données."""
    global running, current_function, time_step, timedata, PFCdata, t
    while True:
        if running:
            
            timedata.append(t)
            if current_function == 0:
                PFCdata.append(np.sin(t))
            elif current_function == 1:
                PFCdata.append(2)




            with fig.batch_update():
                scatter.update(x=timedata, y=PFCdata)
                
            t += time_step
            # time.sleep(0.5)
        
        await asyncio.sleep(time_step)

def start_or_next_function(button):
    """Fonction pour démarrer ou changer la fonction."""
    global running, current_function
    current_function = (current_function + 1) % 2
    running = True

def stop_plot(button):
    """Fonction pour arrêter la mise à jour."""
    global running
    running = False

def reset_plot(button):
    """Fonction pour réinitialiser les données et le temps t."""
    global running, timedata, PFCdata, t
    running = False
    timedata = []
    PFCdata = []
    t = 0
    with fig.batch_update():
        scatter.update(x=timedata, y=PFCdata)

def panne_plot(button):
    """Fonction pour arrêter la mise à jour."""
    global Panne
    Panne = True

def Finpanne_plot(button):
    """Fonction pour arrêter la mise à jour."""
    global Panne
    Panne = False

def create_buttons():
    """Crée les boutons interactifs."""
    start_button = widgets.Button(description="Start/Next Function")
    stop_button = widgets.Button(description="Stop")
    reset_button = widgets.Button(description="Reset")

    panne_button = widgets.Button(description="Panne")
    Finpanne_button = widgets.Button(description="Fin Panne")
    
    panne_button.on_click(panne_plot)
    Finpanne_button.on_click(Finpanne_plot)


    start_button.on_click(start_or_next_function)
    stop_button.on_click(stop_plot)
    reset_button.on_click(reset_plot)

    display(widgets.HBox([start_button, stop_button, reset_button, panne_button, Finpanne_button]), fig)

def main(initial_time_step=1, initial_t=0):
    """Fonction principale pour démarrer l'application."""
    initialize_variables(initial_time_step, initial_t)
    global fig, scatter
    fig, scatter = create_figure()
    create_buttons()
    loop = asyncio.get_event_loop()
    loop.create_task(update_data())

# Appel de la fonction principale pour démarrer l'application
main()


FigureWidget({
    'data': [{'mode': 'lines+markers', 'type': 'scatter', 'uid': '565e1197-7dec-4c38-a90a-ca205c6650e9'}],
    'layout': {'template': '...',
               'title': {'text': 'Courbe Interactive'},
               'xaxis': {'title': {'text': 'Temps'}},
               'yaxis': {'title': {'text': 'Valeur'}}}
})

# Simulation de la Production de Moules

On a 4000 kg de fontes dans le four de Coulée avec un pourcentage de Mg = 0.0045 %.
On consonmme 36 Kg de fonte par minute dans le four de Coulée.
On perd 0.005 % de Mg par minute.

Tu es un mathématicien et tu dois Tracer la courbe d'évolution du Mg
dans le four de Coulée. 

Si tu as des questions ou s'il te manque certaines informations, n'hésite pas à me les demander !!

Soit PFC_Consom, la fonte utiliser pour produire les moules 
Soit Mg, le pourcentage de Mg une  variable qui intervient lors de la production des moules
Que l'on conservera dans un dictionnaire : times_dict, PFC_Consomvalues_dict, Mgvalues_dict

Soit temps_serie, le temps mis pour passer au modeles suivant

Soit PFCmin et Mgmin, la masse de fonte minimal et le pourcentage de Mg minimal
que peuvent prendre PFC_Consom et Mg


A chaque temps t, on a 4 états possibles:
 - Soit on produit des moules 
 - Soit on change de modeles
 - Soit on est en Panne
 - Soit on est à la Fin

Lors que l'on produit des moules :

    Si etat est egal à etat_Consom alors # On verifie que l'on est en etat consommation dans le temps

        si PFC_Consom - cadence_consommation_kg_min et Mg - 0.0005  >= PFCmin et Mgmin alors  # Si l'on peut consommer alors on le fait !!
            
            PFC_Consom -= cadence_consommation_kg_min et Mg -= 0.0005

            # Quantité de moules du i-ème modèle produits en une minute
            cadence_moule_par_minute = cadence_consommation_kg_min / masse_grappe_moule  # en unités/min

            # Quantité de moules restants du i-ème modèle après une minute
            quantite_moules_restants = quantite_moules_a_produire - cadence_moule_par_minute # en unités

            # on met à jour liste_quantites_moules_a_produire et liste_masses_grappes_moules
            si quantite_moules_restants est nul alors 
                - Le temps passe a l'etat changement de Modeles
                - on passe au modele suivant dans les listes : 
                    # Suppression des éléments déjà traités dans les listes
                    liste_masses_grappes_moules.pop(0)
                    liste_quantites_moules_a_produire.pop(0)
                    liste_cadences_moule_par_heure.pop(0)
            sinon 
                - remplacer quantite_moules_a_produire par quantite_moules_restants
                - Le temps reste a l'etat on produit des moules

            si liste_masses_grappes_moules est vide alors 
                - Le temps passe a l'etat Fin ( on arrete toute la proccedure )
                

        Sinon # Si l'on ne peut pas consommer alors on rajoute de la fonte venant de la poche et on consomme
            PFC_Consom += 1250 et Mg = 0.0045 # on rajoute de la fonte venant de la poche 

            # Maintenant on peut consommer !!
            PFC_Consom -= cadence_consommation_kg_min et Mg -= 0.0005

            # Quantité de moules du i-ème modèle produits en une minute
            cadence_moule_par_minute = cadence_consommation_kg_min / masse_grappe_moule  # en unités/min

            # Quantité de moules restants du i-ème modèle après une minute
            quantite_moules_restants = quantite_moules_a_produire - cadence_moule_par_minute # en unités

            # on met à jour liste_quantites_moules_a_produire et liste_masses_grappes_moules
            si quantite_moules_restants est nul alors 
                - Le temps passe a l'etat changement de Modeles
                - on passe au modele suivant dans les listes : 
                    # Suppression des éléments déjà traités dans les listes
                    liste_masses_grappes_moules.pop(0)
                    liste_quantites_moules_a_produire.pop(0)
                    liste_cadences_moule_par_heure.pop(0)
            sinon 
                - remplacer quantite_moules_a_produire par quantite_moules_restants
                - Le temps reste a l'etat on produit des moules

            si liste_masses_grappes_moules est vide alors 
                - Le temps passe a l'etat Fin ( on arrete toute la proccedure )



Lors que l'on change de modeles :

    Si etat est egal à etat_Serie alors # On verifie que l'on est en etat Changement de Serie dans le temps
        Mg -= 0.0005 # Mg perd son pourcentage
        temps_serie -= 1 # On decremente temps_serie de 1   

        Si FC_Consom - cadence_consommation_kg_min et Mg - 0.0005  <= PFCmin et Mgmin alors  # Si l'on ne peut pas consommer alors on ajoute de la fonte !!
            PFC_Consom += 1250 et Mg = 0.0045 # on rajoute de la fonte venant de la poche 
    
        Si PFC_Consom >= PFCMax :
            "Error on est bloqué"

        Si temps_serie est nul alors 
            - Le temps passe à l'état produire des moules
        Sinon 
            - Le temps reste à temps_serie 



Lors que l'on est en Panne :

    Si etat est egal à etat_Panne alors # On verifie que l'on est en etat panne dans le temps
        Mg -= 0.0005 #Mg perd son pourcentage
        temps_Panne -= 1 # On decremente temps_Panne de 1   
        Si temps_Panne est nul alors 
            - Le temps passe à l'état produire des moules
        Sinon 
            - Le temps reste à temps_panne 



Remarques : 
    L'utilisateur doit avoir un bouton pour pouvoir activer l'etat en Panne

In [93]:
# Données initiales
liste_cadences_moule_par_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190]  # en unités/heure
liste_quantites_moules_a_produire = [121, 50, 35, 10, 100, 50, 35, 50, 40]  # en unités
liste_masses_grappes_moules = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65]  # en kg

# Variables de simulation
PFCmin = 2500 # en kg
PFCmax = 5000 # en kg
Mgmin = 0.035 # en %
Mgmax = 0.045 # en %
eC = 0.0005 # en %
poche_i = 1250 # en kg
tmin = 0 # en minutes
t = 0 # en minutes


temps_serie = 10  # Initialisation du temps pour changer de modèle
temps_Panne = 10  # Initialisation du temps pour la panne

# Dictionnaires pour stocker les variables et états
cles = ["chargement_et_fusion", "Four 1", "Four 2", "Four 3", "Four 4", "Four 5"]
times_dict = {cle: [] for cle in cles}
PFC_Consomvalues_dict = {cle: [] for cle in cles}
Mgvalues_dict = {cle: [] for cle in cles}


PFC_Consom = 3750  # Initialisation de la fonte consommée
Mg =  Mgmax  # Initialisation du pourcentage de Mg
tmax = 4*60 # en minutes

# Initialisation des états
etat = "etat_Consom"  # État initial : production de moules

temps_serie_courant = temps_serie
temps_Panne_courant = temps_Panne
numPoche = 3
numFours = 1



#  if  debut < t and  t > fin :
debut,fin = 73,73


def ProduirMoulesParMinute(Mg, PFC_Consom, quantite_moules_a_produire, cadence_consommation_kg_min, cadence_moule_par_minute):
    
    # Mise à jour du pourcentage de Mg et du poids fonte coulée
    PFC_Consom -= cadence_consommation_kg_min
    Mg -= eC

    if PFC_Consom > PFCmax:
        print("On a dépassé la capacité du Four de Coulée")
    

    quantite_moules_restants = quantite_moules_a_produire - cadence_moule_par_minute

    if debut < t < fin:
        print("Ici2", Mg, PFC_Consom, cadence_consommation_kg_min, quantite_moules_a_produire, quantite_moules_restants)

    # On a fini de réaliser le i-ème modèle alors on passe au i+1-ème modèle
    if quantite_moules_restants <= 0:
        etat = "etat_Serie"
        # Suppression des éléments déjà traités dans les listes
        liste_masses_grappes_moules.pop(0)
        liste_quantites_moules_a_produire.pop(0)
        liste_cadences_moule_par_heure.pop(0)

        # Si on a fini de tout produire alors on stoppe la procédure 
        # On passe à l'état Fin
        if not liste_masses_grappes_moules:
            etat = "etat_Fin"

        if debut < t < fin:
            print("Ici3", Mg, PFC_Consom, cadence_consommation_kg_min, quantite_moules_a_produire, quantite_moules_restants)

    # On a fini de réaliser le j-ème moule du i-ème modèle alors on passe au j-ème moule
    # En restant dans l'état consommation 
    else:
        if debut < t < fin:
            print("Ici3", Mg, PFC_Consom, cadence_consommation_kg_min, quantite_moules_a_produire, quantite_moules_restants)

        liste_quantites_moules_a_produire[0] = quantite_moules_restants
        etat = "etat_Consom"
    
    return Mg, PFC_Consom, etat


# Boucle de simulation
while t < tmax :
    if etat == "etat_Consom":
        masse_grappe_moule = liste_masses_grappes_moules[0] 
        cadence_moule_par_heure = liste_cadences_moule_par_heure[0]
        quantite_moules_a_produire  = liste_quantites_moules_a_produire[0]
        
        # La cadence de consommation en kg/min du i-ème modèle
        cadence_consommation_kg_min = cadence_moule_par_heure / 60 * masse_grappe_moule  # en kg/min

        # Quantités de moules du i-ème modèle produits en une minute
        cadence_moule_par_minute = cadence_consommation_kg_min / masse_grappe_moule # en unités/min
        
        if  debut < t and  t < fin :
            print('t = ',t)
            print("Ici1", Mg, PFC_Consom, cadence_consommation_kg_min,quantite_moules_a_produire)
        
        # Si l'on peut produire des moules (c-a-d Mg et PFC sont okay) alors on on le fait !!
        if (PFC_Consom - cadence_consommation_kg_min >= PFCmin) and (Mg - eC >= Mgmin):
            
            Mg, PFC_Consom, etat = ProduirMoulesParMinute(Mg, PFC_Consom, quantite_moules_a_produire, cadence_consommation_kg_min, cadence_moule_par_minute)
                
        else:
            # Ajout poche car Mg ou PFC insuffisant !!
            PFC_Consom += poche_i
            Mg = Mgmax 


            Mg, PFC_Consom, etat = ProduirMoulesParMinute(Mg, PFC_Consom, quantite_moules_a_produire, cadence_consommation_kg_min, cadence_moule_par_minute)
            

            if numPoche < 4:
                numPoche += 1
            else :
                numPoche = 1
                numFours += 1


            if PFC_Consom > PFCmax :
                print("On a depasser pas la capacité du Fours de Coulée")     

    elif etat == "etat_Serie":
        # print("Changement de Serie")
        Mg -= eC
        temps_serie_courant -= 1
        
        # Si le Mg n'est pas bon alors on ajoute de la fonte pour la prochaine serie !! 
        if (PFC_Consom - cadence_consommation_kg_min <= PFCmin) and (Mg - eC <= Mgmin):
            PFC_Consom += poche_i
            Mg = Mgmax
            print("Mg pas bon pendant le changement de Serie") 

        if temps_serie_courant <= 0:
            if  debut < t and  t < fin :
                print("Fin Temps Serie",Mg, PFC_Consom, cadence_consommation_kg_min)
            
            temps_serie_courant = temps_serie
            etat = "etat_Consom"
        else:
            etat == "etat_Serie" 

    elif etat == "etat_Panne":
        Mg -= eC
        temps_Panne_courant -= 1
        
        if temps_Panne_courant <= 0:
            etat = "etat_Consom"
            if  debut < t and  t < fin :
                print("Fin Panne",Mg, PFC_Consom, cadence_consommation_kg_min)
            temps_Panne_courant = temps_Panne
        else:
            etat == "etat_Panne"

    elif etat == "etat_Fin":
        break
    
    # Calcul du delai_avt_traitement et DurreLimite
    DurreLimite = DurreLimite
    delai_avt_traitement = delai_avt_traitement

    

    # Mise à jour du temps
    t += 1
    cle = "Four 1"
    cle = f"Four {numFours}" 
    PFC_Consomvalues_dict[cle].append(PFC_Consom)
    Mgvalues_dict[cle].append(Mg)
    times_dict[cle].append(t)



In [94]:
import plotly.graph_objects as go

# Fonction pour insérer le dernier élément de la clé actuelle au début de la liste de la clé suivante
def inserer_dernier_element(liste_cle, times_dict, PFC_Consomvalues_dict, Mgvalues_dict):
    for i in range(len(liste_cle) - 1):
        derniere_valeur_t = times_dict[liste_cle[i]][-1] if times_dict[liste_cle[i]] else None
        derniere_valeur_PFC = PFC_Consomvalues_dict[liste_cle[i]][-1] if PFC_Consomvalues_dict[liste_cle[i]] else None
        derniere_valeur_Mg = Mgvalues_dict[liste_cle[i]][-1] if Mgvalues_dict[liste_cle[i]] else None
        if derniere_valeur_t is not None:
            times_dict[liste_cle[i + 1]] = [derniere_valeur_t] + times_dict[liste_cle[i + 1]]
            PFC_Consomvalues_dict[liste_cle[i + 1]] = [derniere_valeur_PFC] + PFC_Consomvalues_dict[liste_cle[i + 1]]
            Mgvalues_dict[liste_cle[i + 1]] = [derniere_valeur_Mg] + Mgvalues_dict[liste_cle[i + 1]]


def Plot_consom(PFC_values_dict, Mgvalues_dict, times_dict, PFCmin, PFCmax, Mgmin, Mgmax):
    # Dictionnaire de couleurs pour chaque clé
    color_dict = {
        "chargement_et_fusion": 'royalblue',
        "Four 1": 'firebrick',
        "Four 2": 'green',
        "Four 3": 'orange',
        "Four 4": 'purple',
        "Four 5": 'royalblue'
    }


    # Initialisation des dictionnaires
    cles = ["chargement_et_fusion", "Four 1", "Four 2", "Four 3", "Four 4", "Four 5"]

    # Appliquer la fonction aux trois dictionnaires
    inserer_dernier_element(cles, times_dict, PFC_Consomvalues_dict, Mgvalues_dict)


    # Create Plotly figure for PFC
    fig_pfc = go.Figure()

    # Add PFC lines for each key
    for key in PFC_values_dict.keys():
        fig_pfc.add_trace(go.Scatter(x=times_dict[key], y=PFC_values_dict[key], mode='lines', name=f'PFC over time ({key})', line=dict(color=color_dict[key])))

    # Add horizontal lines for PFCmin and PFCmax
    tmin, tmax = times_dict["Four 1"][0],times_dict["Four 5"][-1]
    fig_pfc.add_shape(type="line", x0=tmin, y0=PFCmin, x1=tmax, y1=PFCmin, line=dict(color="firebrick", width=2, dash="dash"), name="PFCmin")
    fig_pfc.add_shape(type="line", x0=tmin, y0=PFCmax, x1=tmax, y1=PFCmax, line=dict(color="green", width=2, dash="dash"), name="PFCmax")

    # Customize layout for PFC figure
    fig_pfc.update_layout(title='Evolution de la fonte dans le Four de Coulée au cours du temps (PFC)', 
                          xaxis_title='Time (minutes)', yaxis_title='PFC (kg)', 
                          template='plotly', showlegend=True,
                          annotations=[
                              dict(x=tmax, y=PFCmin, xref="x", yref="y", text="PFCmin", showarrow=True, arrowhead=2, ax=20, ay=-30),
                              dict(x=tmax, y=PFCmax, xref="x", yref="y", text="PFCmax", showarrow=True, arrowhead=2, ax=20, ay=-30)
                          ])

    # Show the PFC plot
    fig_pfc.show()

    # Create Plotly figure for Mg
    fig_mg = go.Figure()

    # Add Mg lines for each key
    for key in Mgvalues_dict.keys():
        fig_mg.add_trace(go.Scatter(x=times_dict[key], y=Mgvalues_dict[key], mode='lines', name=f'Mg over time ({key})', line=dict(color=color_dict[key])))

    # Add horizontal lines for Mgmin and Mgmax
    fig_mg.add_shape(type="line", x0=tmin, y0=Mgmin, x1=tmax, y1=Mgmin, line=dict(color="firebrick", width=2, dash="dash"), name="Mgmin")
    fig_mg.add_shape(type="line", x0=tmin, y0=Mgmax, x1=tmax, y1=Mgmax, line=dict(color="green", width=2, dash="dash"), name="Mgmax")

    # Customize layout for Mg figure
    fig_mg.update_layout(title='Evolution de la fonte dans le Four de Coulée au cours du temps (Mg)', 
                         xaxis_title='Time (minutes)', yaxis_title='Mg (kg)', 
                         template='plotly', showlegend=True,
                         annotations=[
                              dict(x=tmax, y=Mgmin, xref="x", yref="y", text="Mgmin", showarrow=True, arrowhead=2, ax=20, ay=-30),
                              dict(x=tmax, y=Mgmax, xref="x", yref="y", text="Mgmax", showarrow=True, arrowhead=2, ax=20, ay=-30)
                          ])

    # Show the Mg plot
    fig_mg.show()

# Call the function with the appropriate dictionaries and parameters
Plot_consom(PFC_Consomvalues_dict, Mgvalues_dict, times_dict, PFCmin, PFCmax, Mgmin, Mgmax)


# Autres Brouillon


Tu es un mathématicien et du dois effectuer cette tache suivant :

Faire varier la fonction Consomation_fonteCoulee au cours du temps en min
En conservant le temps t, La consomation PFC_Consom et 
le pourcentage de Mg dans un dictionnaire : times_dict, PFC_Consomvalues_dict, Mgvalues_dict
On souhaite interompre la consomation avec une variable la Panne :temps_Panne.
Pendant la Panne : le pourcentage de Mg continue à varier.
Cependant la panne peut intervenir à n'importe quel momment dans le temps et peut etre recurent, 
c'est un bouton que l'utilisateurs active ou non en donnant la durrée de la Panne temps_Panne


n'hésite pas a modifier  Consomation_fonteCoulee pour repondre à ses besoins !!!
Si tu as des questions ou s'il te manque certaines informations, n'hésite pas à me les demander !!
